In [1]:
# Imports
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from tempfile import TemporaryDirectory

import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(42)


Running on device: CUDA


Get data

In [2]:

# CIFAR 10 dataset

# Transformations
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:05<00:00, 29239963.77it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:

# Show different classes
classes = np.unique(np.array(train_set.targets))
classes

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
# Choose random forget indecies from some class
# Index of class
class_index = 1 # cars
class_set = np.where(np.array(train_set.targets) == 1)[0]

# Percantage of whole data ( from class )
amount = 0.01 # 1 %
amount_int = class_set.shape[0] * amount

# Get indeces
forget_idx = np.random.choice(class_set, int(amount_int))

# construct indices of retain from those of the forget set
forget_mask = np.zeros(len(train_set.targets), dtype=bool)
forget_mask[forget_idx] = True
retain_idx = np.arange(forget_mask.size)[~forget_mask]

# split train set into a forget and a retain set
forget_set = torch.utils.data.Subset(train_set, forget_idx)
retain_set = torch.utils.data.Subset(train_set, retain_idx)

# Generate forget and retain loaders
forget_loader = torch.utils.data.DataLoader(
    forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
)
retain_loader = torch.utils.data.DataLoader(
    retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
)

In [5]:

# Add helping dicts


dataloaders = {
    "train": train_loader,
    "val": test_loader
}

dataset_sizes = {"train": len(train_set), "val": len(test_set)}

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_loss = 1000

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(DEVICE)
                    labels = labels.to(DEVICE)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_loss < best_loss:
                    best_loss = epoch_loss
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val loss: {best_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model


In [7]:
# Train ResNet18

model_train = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
model_train = model_train.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_train.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [8]:
# Train model Resnet18 21 epochs
model_train = train_model(model_train, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=21)

Epoch 0/20
----------
train Loss: 1.3891 Acc: 0.5002
val Loss: 1.2895 Acc: 0.5573

Epoch 1/20
----------
train Loss: 0.9827 Acc: 0.6525
val Loss: 1.0393 Acc: 0.6381

Epoch 2/20
----------
train Loss: 0.7896 Acc: 0.7211
val Loss: 0.9406 Acc: 0.6763

Epoch 3/20
----------
train Loss: 0.6621 Acc: 0.7682
val Loss: 0.8359 Acc: 0.7128

Epoch 4/20
----------
train Loss: 0.5501 Acc: 0.8067
val Loss: 0.8857 Acc: 0.7093

Epoch 5/20
----------
train Loss: 0.4535 Acc: 0.8400
val Loss: 1.0570 Acc: 0.6724

Epoch 6/20
----------
train Loss: 0.3747 Acc: 0.8674
val Loss: 0.9209 Acc: 0.7118

Epoch 7/20
----------
train Loss: 0.1564 Acc: 0.9526
val Loss: 0.7859 Acc: 0.7675

Epoch 8/20
----------
train Loss: 0.0833 Acc: 0.9772
val Loss: 0.8564 Acc: 0.7652

Epoch 9/20
----------
train Loss: 0.0484 Acc: 0.9887
val Loss: 0.9579 Acc: 0.7648

Epoch 10/20
----------
train Loss: 0.0275 Acc: 0.9946
val Loss: 1.0741 Acc: 0.7625

Epoch 11/20
----------
train Loss: 0.0144 Acc: 0.9979
val Loss: 1.2268 Acc: 0.7594

Ep

In [9]:

# Save model weights
torch.save(model_train.state_dict(), "model_train_params_best_loss.pt")

In [10]:
# Check accuracy

def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total


print(f"Train set accuracy: {100.0 * accuracy(model_train, train_loader):0.1f}%")
print(f"Test set accuracy: {100.0 * accuracy(model_train, test_loader):0.1f}%")

Train set accuracy: 98.0%
Test set accuracy: 76.8%


In [11]:
# Unlearning algorithm
def unlearning(net, retain, forget, validation):
    """Unlearning by fine-tuning.

    Fine-tuning is a very simple algorithm that trains using only
    the retain set.

    Args:
      net : nn.Module.
        pre-trained model to use as base of unlearning.
      retain : torch.utils.data.DataLoader.
        Dataset loader for access to the retain set. This is the subset
        of the training set that we don't want to forget.
      forget : torch.utils.data.DataLoader.
        Dataset loader for access to the forget set. This is the subset
        of the training set that we want to forget. This method doesn't
        make use of the forget set.
      validation : torch.utils.data.DataLoader.
        Dataset loader for access to the validation set. This method doesn't
        make use of the validation set.
    Returns:
      net : updated model
    """
    epochs = 5

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    net.train()

    for _ in range(epochs):
        for inputs, targets in retain:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        scheduler.step()

    net.eval()
    return net

In [12]:

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    criterion = nn.CrossEntropyLoss(reduction="none")
    all_losses = []

    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        logits = net(inputs)
        losses = criterion(logits, targets).numpy(force=True)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

In [13]:
# MIA score


def simple_mia(sample_loss, members, n_splits=10, random_state=0):
    """Computes cross-validation score of a membership inference attack.

    Args:
      sample_loss : array_like of shape (n,).
        objective function evaluated on n samples.
      members : array_like of shape (n,),
        whether a sample was used for training.
      n_splits: int
        number of splits to use in the cross-validation.
    Returns:
      scores : array_like of size (n_splits,)
    """

    unique_members = np.unique(members)
    if not np.all(unique_members == np.array([0, 1])):
        raise ValueError("members should only have 0 and 1s")

    attack_model = linear_model.LogisticRegression()
    cv = model_selection.StratifiedShuffleSplit(
        n_splits=n_splits, random_state=random_state
    )
    return model_selection.cross_val_score(
        attack_model, sample_loss, members, cv=cv, scoring="accuracy"
    )

In [14]:
# Unlearning algorithm
def unlearning_by_epochs(net, retain, forget, validation, epochs=5):
    """Unlearning by fine-tuning.

    Fine-tuning is a very simple algorithm that trains using only
    the retain set.

    Args:
      net : nn.Module.
        pre-trained model to use as base of unlearning.
      retain : torch.utils.data.DataLoader.
        Dataset loader for access to the retain set. This is the subset
        of the training set that we don't want to forget.
      forget : torch.utils.data.DataLoader.
        Dataset loader for access to the forget set. This is the subset
        of the training set that we want to forget. This method doesn't
        make use of the forget set.
      validation : torch.utils.data.DataLoader.
        Dataset loader for access to the validation set. This method doesn't
        make use of the validation set.
    Returns:
      net : updated model
    """

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    net.train()

    for _ in range(epochs):
        for inputs, targets in retain:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        scheduler.step()

    net.eval()
    return net

In [15]:
train_losses = compute_losses(model_train, train_loader)
test_losses = compute_losses(model_train, test_loader)

# TRAIN MODEL
# Get random test samples
randomize = np.arange(len(test_losses))
np.random.shuffle(randomize)

# Compute forget losses on train dataset
forget_losses = compute_losses(model_train, forget_loader)

# Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
test_losses = test_losses[randomize][: len(forget_losses)]

samples_mia = np.concatenate((test_losses, forget_losses)).reshape((-1, 1))
labels_mia = [0] * len(test_losses) + [1] * len(forget_losses)

mia_scores = simple_mia(samples_mia, labels_mia)

print(
    f"The MIA has an accuracy of {mia_scores.mean():.3f} on forgotten vs unseen images"
)


The MIA has an accuracy of 0.720 on forgotten vs unseen images


In [16]:
print("----- 1 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes

# Show different classes
classes = np.unique(np.array(train_set.targets))

MIA_scores = []
MIA_scores_retain = []
Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []
AD_score = []

for class_num in classes:
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load("model_train_params_best_loss.pt"))
  model_forget_ft.to(DEVICE)

  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount = 0.01 # 1 %
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  # Retain model
  # Add helping dicts
  # MIA

  # Compute forget losses for forget model
  forget_losses_fr = compute_losses(model_ft_forget, forget_loader)
  test_losses_fr = compute_losses(model_ft_forget, test_loader)

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  test_losses_fr = test_losses_fr[randomize][: len(forget_losses)]

  # make sure we have a balanced dataset for the MIA
  samples_mia_fr = np.concatenate((test_losses_fr, forget_losses_fr)).reshape((-1, 1))
  labels_mia_fr = [0] * len(test_losses_fr) + [1] * len(forget_losses_fr)

  mia_scores_fr = simple_mia(samples_mia_fr, labels_mia_fr)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr.mean():.3f} on forgotten vs unseen images on FORGET model"
  )

MIA_scores = [m.mean() for m in MIA_scores]
print(MIA_scores)
print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

# Mean MIA
mean_mia_1 = sum(MIA_scores) / len(MIA_scores)

# Mean Accuracy Retain
mean_accuracy_retain = sum(Accuracy_retain) / len(Accuracy_retain)
mean_accuracy_test = sum(Accuracy_test) / len(Accuracy_test)

# Mean Accuracy forget
mean_accuracy_forget = sum(Accuracy_forget) / len(Accuracy_forget)

print("MEANS")

print(mean_mia_1)
print(mean_accuracy_retain)
print(mean_accuracy_forget)
print(mean_accuracy_test)

----- 1 PERCENT -------
Retain set accuracy FORGET model: 99.4%
Test set accuracy FORGET model: 76.3%
Forget set accuracy FORGET model: 90.0%
The MIA has an accuracy of 0.560 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.7%
Test set accuracy FORGET model: 76.3%
Forget set accuracy FORGET model: 94.0%
The MIA has an accuracy of 0.610 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.7%
Test set accuracy FORGET model: 77.0%
Forget set accuracy FORGET model: 78.0%
The MIA has an accuracy of 0.560 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.5%
Test set accuracy FORGET model: 76.2%
Forget set accuracy FORGET model: 76.0%
The MIA has an accuracy of 0.420 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 76.2%
Forget set accuracy FORGET model: 92.0%
The MIA has an accuracy of 0.520 on forgotten vs unseen images on FORG

In [17]:
print("---- 2 PERCANTAGES -----")
# Generalisation algorithm
# Calculating score across 10 different classes

# Show different classes
classes = np.unique(np.array(train_set.targets))

MIA_scores = []
MIA_scores_retain = []
Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []
AD_score = []

for class_num in classes:
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load("model_train_params_best_loss.pt"))
  model_forget_ft.to(DEVICE)

  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount = 0.02 # 1 %
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  # Retain model
  # Add helping dicts
  # MIA

  # Compute forget losses for forget model
  forget_losses_fr = compute_losses(model_ft_forget, forget_loader)
  test_losses_fr = compute_losses(model_ft_forget, test_loader)

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  test_losses_fr = test_losses_fr[randomize][: len(forget_losses)]

  # make sure we have a balanced dataset for the MIA
  samples_mia_fr = np.concatenate((test_losses_fr, forget_losses_fr)).reshape((-1, 1))
  labels_mia_fr = [0] * len(test_losses_fr) + [1] * len(forget_losses_fr)

  mia_scores_fr = simple_mia(samples_mia_fr, labels_mia_fr)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr.mean():.3f} on forgotten vs unseen images on FORGET model"
  )

MIA_scores = [m.mean() for m in MIA_scores]
print(MIA_scores)
print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

# Mean MIA
mean_mia_1 = sum(MIA_scores) / len(MIA_scores)

# Mean Accuracy Retain
mean_accuracy_retain = sum(Accuracy_retain) / len(Accuracy_retain)
mean_accuracy_test = sum(Accuracy_test) / len(Accuracy_test)

# Mean Accuracy forget
mean_accuracy_forget = sum(Accuracy_forget) / len(Accuracy_forget)

print("MEANS")

print(mean_mia_1)
print(mean_accuracy_retain)
print(mean_accuracy_forget)
print(mean_accuracy_test)

---- 2 PERCANTAGES -----
Retain set accuracy FORGET model: 99.7%
Test set accuracy FORGET model: 76.3%
Forget set accuracy FORGET model: 86.0%
The MIA has an accuracy of 0.753 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.2%
Test set accuracy FORGET model: 76.4%
Forget set accuracy FORGET model: 92.0%
The MIA has an accuracy of 0.687 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 76.7%
Forget set accuracy FORGET model: 80.0%
The MIA has an accuracy of 0.667 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 76.8%
Forget set accuracy FORGET model: 74.0%
The MIA has an accuracy of 0.667 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.9%
Test set accuracy FORGET model: 76.8%
Forget set accuracy FORGET model: 84.0%
The MIA has an accuracy of 0.700 on forgotten vs unseen images on FOR

In [20]:
print("---- 5 PERCANTAGES -----")
# Generalisation algorithm
# Calculating score across 10 different classes

# Show different classes
classes = np.unique(np.array(train_set.targets))

MIA_scores = []
MIA_scores_retain = []
Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []
AD_score = []

for class_num in classes:
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load("model_train_params_best_loss.pt"))
  model_forget_ft.to(DEVICE)

  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount = 0.05 # 1 %
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  # Retain model
  # Add helping dicts
  # MIA

  # Compute forget losses for forget model
  forget_losses_fr = compute_losses(model_ft_forget, forget_loader)
  test_losses_fr = compute_losses(model_ft_forget, test_loader)

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  test_losses_fr = test_losses_fr[randomize][: len(forget_losses)]

  # make sure we have a balanced dataset for the MIA
  samples_mia_fr = np.concatenate((test_losses_fr, forget_losses_fr)).reshape((-1, 1))
  labels_mia_fr = [0] * len(test_losses_fr) + [1] * len(forget_losses_fr)

  mia_scores_fr = simple_mia(samples_mia_fr, labels_mia_fr)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr.mean():.3f} on forgotten vs unseen images on FORGET model"
  )

MIA_scores = [m.mean() for m in MIA_scores]
print(MIA_scores)
print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

# Mean MIA
mean_mia_1 = sum(MIA_scores) / len(MIA_scores)

# Mean Accuracy Retain
mean_accuracy_retain = sum(Accuracy_retain) / len(Accuracy_retain)
mean_accuracy_test = sum(Accuracy_test) / len(Accuracy_test)

# Mean Accuracy forget
mean_accuracy_forget = sum(Accuracy_forget) / len(Accuracy_forget)

print("MEANS")

print(mean_mia_1)
print(mean_accuracy_retain)
print(mean_accuracy_forget)
print(mean_accuracy_test)

---- 5 PERCANTAGES -----
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 76.9%
Forget set accuracy FORGET model: 87.2%
The MIA has an accuracy of 0.833 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 77.0%
Forget set accuracy FORGET model: 92.0%
The MIA has an accuracy of 0.830 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.3%
Test set accuracy FORGET model: 76.1%
Forget set accuracy FORGET model: 76.0%
The MIA has an accuracy of 0.833 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.9%
Test set accuracy FORGET model: 77.2%
Forget set accuracy FORGET model: 74.0%
The MIA has an accuracy of 0.833 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.7%
Test set accuracy FORGET model: 76.6%
Forget set accuracy FORGET model: 84.4%
The MIA has an accuracy of 0.833 on forgotten vs unseen images on FOR

In [21]:
print("---- 10 % -----")
# Generalisation algorithm
# Calculating score across 10 different classes

# Show different classes
classes = np.unique(np.array(train_set.targets))

MIA_scores = []
MIA_scores_retain = []
Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []
AD_score = []

for class_num in classes:
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load("model_train_params_best_loss.pt"))
  model_forget_ft.to(DEVICE)

  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount = 0.1 # 1 %
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  # Retain model
  # Add helping dicts
  # MIA

  # Compute forget losses for forget model
  forget_losses_fr = compute_losses(model_ft_forget, forget_loader)
  test_losses_fr = compute_losses(model_ft_forget, test_loader)

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  test_losses_fr = test_losses_fr[randomize][: len(forget_losses)]

  # make sure we have a balanced dataset for the MIA
  samples_mia_fr = np.concatenate((test_losses_fr, forget_losses_fr)).reshape((-1, 1))
  labels_mia_fr = [0] * len(test_losses_fr) + [1] * len(forget_losses_fr)

  mia_scores_fr = simple_mia(samples_mia_fr, labels_mia_fr)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr.mean():.3f} on forgotten vs unseen images on FORGET model"
  )

MIA_scores = [m.mean() for m in MIA_scores]
print(MIA_scores)
print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

# Mean MIA
mean_mia_1 = sum(MIA_scores) / len(MIA_scores)

# Mean Accuracy Retain
mean_accuracy_retain = sum(Accuracy_retain) / len(Accuracy_retain)
mean_accuracy_test = sum(Accuracy_test) / len(Accuracy_test)

# Mean Accuracy forget
mean_accuracy_forget = sum(Accuracy_forget) / len(Accuracy_forget)

print("MEANS")

print(mean_mia_1)
print(mean_accuracy_retain)
print(mean_accuracy_forget)
print(mean_accuracy_test)

---- 10 % -----
Retain set accuracy FORGET model: 99.9%
Test set accuracy FORGET model: 77.0%
Forget set accuracy FORGET model: 87.8%
The MIA has an accuracy of 0.911 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.9%
Test set accuracy FORGET model: 76.9%
Forget set accuracy FORGET model: 90.2%
The MIA has an accuracy of 0.916 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 76.2%
Forget set accuracy FORGET model: 81.2%
The MIA has an accuracy of 0.909 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.8%
Test set accuracy FORGET model: 76.7%
Forget set accuracy FORGET model: 74.2%
The MIA has an accuracy of 0.909 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model: 99.7%
Test set accuracy FORGET model: 76.8%
Forget set accuracy FORGET model: 80.2%
The MIA has an accuracy of 0.909 on forgotten vs unseen images on FORGET model

In [22]:
# Generalisation algorithm
# Calculating score across 10 different classes

# Show different classes
classes = np.unique(np.array(train_set.targets))

MIA_scores = []
MIA_scores_retain = []
Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []
AD_score = []
# Define model from trained params

model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
model_forget_ft.load_state_dict(torch.load("model_train_params_best_loss.pt"))
model_forget_ft.to(DEVICE)

for epoch_num in range(10):


  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == 1)[0]

  # Percantage of whole data ( from class )
  amount = 0.02 # 2 %
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # Unlearn
  model_ft_forget = unlearning_by_epochs(model_forget_ft, retain_loader, forget_loader, test_loader, 1)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model epoch {epoch_num}: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model epoch {epoch_num}: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model {epoch_num}:: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")
  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))

  # Retain model
  # Add helping dicts
  # MIA

  # Compute forget losses for forget model
  forget_losses_fr = compute_losses(model_ft_forget, forget_loader)
  test_losses_fr = compute_losses(model_ft_forget, test_loader)

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  test_losses_fr = test_losses_fr[randomize][: len(forget_losses)]

  # make sure we have a balanced dataset for the MIA
  samples_mia_fr = np.concatenate((test_losses_fr, forget_losses_fr)).reshape((-1, 1))
  labels_mia_fr = [0] * len(test_losses_fr) + [1] * len(forget_losses_fr)

  mia_scores_fr = simple_mia(samples_mia_fr, labels_mia_fr)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr.mean():.3f} on forgotten vs unseen images on FORGET model"
  )

MIA_scores = [m.mean() for m in MIA_scores]
print(MIA_scores)
print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

# Mean MIA
mean_mia_1 = sum(MIA_scores) / len(MIA_scores)

# Mean Accuracy Retain
mean_accuracy_retain = sum(Accuracy_retain) / len(Accuracy_retain)
mean_accuracy_test = sum(Accuracy_test) / len(Accuracy_test)

# Mean Accuracy forget
mean_accuracy_forget = sum(Accuracy_forget) / len(Accuracy_forget)

print("MEANS")

print(mean_mia_1)
print(mean_accuracy_retain)
print(mean_accuracy_forget)
print(mean_accuracy_test)

Retain set accuracy FORGET model epoch 0: 81.6%
Test set accuracy FORGET model epoch 0: 70.8%
Forget set accuracy FORGET model 0:: 81.0%
The MIA has an accuracy of 0.667 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model epoch 1: 85.1%
Test set accuracy FORGET model epoch 1: 72.0%
Forget set accuracy FORGET model 1:: 95.0%
The MIA has an accuracy of 0.767 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model epoch 2: 87.0%
Test set accuracy FORGET model epoch 2: 71.5%
Forget set accuracy FORGET model 2:: 81.0%
The MIA has an accuracy of 0.667 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model epoch 3: 89.2%
Test set accuracy FORGET model epoch 3: 72.9%
Forget set accuracy FORGET model 3:: 85.0%
The MIA has an accuracy of 0.667 on forgotten vs unseen images on FORGET model
Retain set accuracy FORGET model epoch 4: 87.8%
Test set accuracy FORGET model epoch 4: 71.9%
Forget set accuracy FORGET model 4:: 83.0%